In [1]:
import pandas as pd
import pandas as pd
import pandas as pd
import nltk
nltk.download('punkt') 
from nltk.tokenize import word_tokenize
import fitz  
import requests
import os
import time

/tmp/ipykernel_9432/3111190662.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
%pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 6.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
with open("/workspaces/Wikistim-Summarization/my_entrez_apikey.txt", 'r') as file:
    ENTREZ_APIKEY = file.read().strip()

In [19]:
from Bio import Entrez
from bs4 import BeautifulSoup
import pandas as pd

def search_pubmed_for_fulltext(keyword):
    Entrez.email = "suzannathompson59@gmail.com" 
    Entrez.api_key = ENTREZ_APIKEY

    try:
        # Search PubMed for articles with PMC identifiers
        search_handle = Entrez.esearch(db="pubmed", term=f"{keyword} AND pmc cc license[filter]", retmax=10)
        search_results = Entrez.read(search_handle)
        search_handle.close()

        id_list = search_results["IdList"]

        if not id_list:
            print("No articles found for the given search criteria.")
            return []

        pmc_ids = []
        # Fetch details of the publications to extract PMC links
        fetch_handle = Entrez.efetch(db="pubmed", id=",".join(id_list), retmode="xml")
        articles = Entrez.read(fetch_handle)
        fetch_handle.close()

        for article in articles["PubmedArticle"]:
            pmc_id = [data for data in article['PubmedData']['ArticleIdList'] if data.attributes['IdType'] == 'pmc']
            if pmc_id:
                pmc_id = pmc_id[0]  # Assuming the first PMC ID is the relevant one
                pmc_ids.append(pmc_id)
        
        return pmc_ids

    except Exception as e:
        print(f"An error occurred: {e}")
        return []


def download_full_text_pmc(pmc_ids):
    """Download the full text for a list of PMC IDs."""
    full_texts = []
    for pmc_id in pmc_ids:
        try:
            handle = Entrez.efetch(db="pmc", id=pmc_id, rettype="xml")
            xml_data = handle.read()
            handle.close()
            full_texts.append(xml_data)
        except Exception as e:
            print(f"Failed to download PMC ID {pmc_id}: {e}")
    return full_texts

# Function to parse a single XML string
def parse_xml(xml_data):
    soup = BeautifulSoup(xml_data, 'xml')  # Parse each XML string with BeautifulSoup
    
    # Extracting the article's title
    title = soup.find('article-title').text if soup.find('article-title') else 'No Title Found'
    
    # Extracting the publication date
    pub_date = soup.find('pub-date')
    if pub_date:
        year = pub_date.find('year').text if pub_date.find('year') else 'No Year'
        month = pub_date.find('month').text if pub_date.find('month') else 'No Month'
        day = pub_date.find('day').text if pub_date.find('day') else 'No Day'
        pub_date = f"{year}-{month}-{day}"
    else:
        pub_date = "No Publication Date Found"
    
    # Extracting authors
    authors = []
    for contrib in soup.find_all('contrib', attrs={'contrib-type': 'author'}):
        surname = contrib.find('surname').text if contrib.find('surname') else ''
        given_names = contrib.find('given-names').text if contrib.find('given-names') else ''
        authors.append(f"{given_names} {surname}")
    
    authors_str = ', '.join(authors) if authors else 'No Authors Found'
    
    return title, pub_date, authors_str


def parse_article_text(xml_data):
    soup = BeautifulSoup(xml_data, 'xml')
    body = soup.find('body')
    if not body:
        return "No body found in the article", {}

    # Extract sections and text
    sections = {}
    full_text = ""
    for sec in body.find_all('sec'):
        # Extract section title
        sec_title = sec.find('title').text if sec.find('title') else 'No Title'
        
        # Extract paragraphs within the section
        paragraphs = [p.text for p in sec.find_all('p')]
        sec_text = "\n".join(paragraphs)
        
        # Store the section text
        sections[sec_title] = sec_text
        
        # Append to full text
        full_text += f"{sec_title}\n{sec_text}\n\n"

    return full_text, sections




/tmp/ipykernel_27061/571296758.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [17]:
# Example usage
keyword = "Deep Brain Stimulation"
keyword_articles = search_pubmed_for_fulltext(keyword)
print( "Number of articles found with keyword: ", len(keyword_articles))
if keyword_articles:
    for art in keyword_articles:
        print(art)
else:
    print("No articles found with full text links.")

Number of articles found with keyword:  10
PMC10858876
PMC10826894
PMC10856385
PMC10849442
PMC10853182
PMC10847241
PMC10851535
PMC10844373
PMC10833010
PMC10837616


In [23]:
# Prepare data for DataFrame
data_for_df = []

full_texts = download_full_text_pmc(keyword_articles[:1])  # Limit to first 5 for demonstration
print(f"Downloaded {len(full_texts)} full-text articles.")


for xml_data in full_texts:
    title, pub_date, authors = parse_xml(xml_data)
    full_text, _ = parse_article_text(xml_data)  # Assuming we only need the full_text here
    data_for_df.append((title, pub_date, authors, full_text))

# Create DataFrame
df = pd.DataFrame(data_for_df, columns=['Title', 'Publication Date', 'Authors', 'Full Text'])

df['Full Text'][0]

Downloaded 1 full-text articles.


'Introduction\nEpilepsy is a serious global burden on public health. This chronic and debilitating neurological disorder affects people of all ages, with an estimated 50 million sufferers worldwide. Aside from recurrent seizures, epilepsy is commonly associated with comorbidities such as cognitive, emotional, and sleep disorders1–5. The main treatment for epilepsy is drug therapy, however, a significant proportion of patients do not respond to drugs6,7. Anti-seizure drugs typically target widely-expressed ion channels and neurotransmitters, causing general suppression of brain activity8. This has additional limitations, including the development of drug resistance, adverse and unpredictable cognitive side effects, slow onset and offset of drug action, and patient non-compliance9–15. For these reasons, more specific and rapid seizure control, through electrical modulation of specific brain regions with implanted electrodes (deep brain stimulation, DBS), is seen as a promising alternativ

## Import data

In [6]:
raw_data = pd.read_csv('/workspaces/Wikistim-Summarization/00_source_data/completed_papers_wikistim.csv')
data_with_url = raw_data[~raw_data['Full text link'].isnull()]

## Download PDFS from url

In [7]:
download_dir = "/workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs"
os.makedirs(download_dir, exist_ok=True)


user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

max_retries = 3

data_with_url['Downloaded PDF Path'] = ""


for idx, url in data_with_url['Full text link'].items():
    retry_count = 0
    while retry_count < max_retries:
        try:
            headers = {"User-Agent": user_agent}
            response = requests.get(url, headers=headers, timeout=30)
            
            if response.status_code == 200:
                filename = f"pdf_{idx}.pdf"
                file_path = os.path.join(download_dir, filename)
                with open(file_path, 'wb') as pdf_file:
                    pdf_file.write(response.content)
                data_with_url.at[idx, 'Downloaded PDF Path'] = file_path
                
                print(f"Downloaded PDF {idx} to {file_path}")
                break
            else:
                print(f"Failed to download PDF {idx} from URL {url}. Status Code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading PDF {idx} from URL {url}: {e}")

        retry_count += 1
        time.sleep(5)  

    if retry_count >= max_retries:
        print(f"Maximum retry count reached for PDF {idx}. Download failed.")

successful_downloads = data_with_url['Downloaded PDF Path'].count()

total_papers = len(data_with_url)

success_percentage = (successful_downloads / total_papers) * 100

print(f"Successfully downloaded {successful_downloads} out of {total_papers} papers ({success_percentage:.2f}% success rate)")


/tmp/ipykernel_5232/3820561139.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_with_url['Downloaded PDF Path'] = ""


Downloaded PDF 0 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_0.pdf
Downloaded PDF 1 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_1.pdf
Downloaded PDF 3 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_3.pdf
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Maximum retry count reached for PDF 4. Download failed.
Downloaded PDF 5 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_5.pdf
Downloaded PDF 6 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_6.pdf
Failed to download PDF 7 from URL https://www.jpsmjournal.com/article/S0885-3924(96)00322-3/pdf. Status Code: 403
Failed to download P

In [16]:
successful_downloads = 0  # Initialize a counter for successful downloads

for idx, url in data_with_url['Full text link'].items():
    retry_count = 0
    while retry_count < max_retries:
        try:
            headers = {"User-Agent": user_agent}
            response = requests.get(url, headers=headers, timeout=30)
            
            if response.status_code == 200:
                # Define the filename based on the index
                filename = f"pdf_{idx}.pdf"
                # Define the full path to save the file
                file_path = os.path.join(download_dir, filename)
                
                # Save the PDF to the specified directory with the indexed filename
                with open(file_path, 'wb') as pdf_file:
                    pdf_file.write(response.content)
                
                # Update the DataFrame with the downloaded PDF path
                data_with_url.at[idx, 'Downloaded PDF Path'] = file_path
                
                print(f"Downloaded PDF {idx} to {file_path}")
                successful_downloads += 1  # Increment the counter for successful downloads
                break  # Successful download, exit retry loop
            else:
                print(f"Failed to download PDF {idx} from URL {url}. Status Code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading PDF {idx} from URL {url}: {e}")
        
        # Increment the retry count and wait before the next retry
        retry_count += 1
        time.sleep(5)  # Wait for 5 seconds before retrying

    if retry_count >= max_retries:
        print(f"Maximum retry count reached for PDF {idx}. Download failed.")

# Calculate the success rate
success_rate = (successful_downloads / len(data_with_url)) * 100

# Print the result
print(f"Download success rate: {success_rate:.2f}%")


Downloaded PDF 0 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_0.pdf
Downloaded PDF 1 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_1.pdf
Downloaded PDF 3 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_3.pdf
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Maximum retry count reached for PDF 4. Download failed.
Downloaded PDF 5 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_5.pdf
Downloaded PDF 6 to /workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_6.pdf
Failed to download PDF 7 from URL https://www.jpsmjournal.com/article/S0885-3924(96)00322-3/pdf. Status Code: 403
Failed to download P

## Scrape text from downloaded pdfs

In [17]:
data_with_url['full_article_text'] = ""


for idx, row in data_with_url.iterrows():
    pdf_filename = row['Downloaded PDF Path']
    if pdf_filename and pdf_filename.endswith(".pdf"):  # Check if PDF filename is not empty and ends with .pdf
        pdf_path = f"/{pdf_filename}"  # Add the prefix to the PDF path
        try:
            doc = fitz.open(pdf_path)
            # Extract text from all pages and join them into a single string
            full_text = ""
            for page in doc:
                full_text += page.get_text()
            
            # Check if the extracted text is truly empty (no visible characters)
            if full_text.strip():
                # Update the DataFrame with the extracted text
                data_with_url.at[idx, 'full_article_text'] = full_text
            else:
                print(f"Skipped processing {pdf_path} because extracted text is empty.")
        except Exception as e:
            print(f"Skipped processing {pdf_path} due to an error: {e}")
            continue  # Skip this document and move to the next one

# Filter out rows where 'full_article_text' is truly empty
filtered_data = data_with_url[data_with_url['full_article_text'].str.strip() != ""]

# Calculate the number of successfully scraped texts
successful_scrapes = filtered_data.shape[0]

# Calculate the total number of papers
total_papers = len(data_with_url)

# Calculate the percentage of successful scrapes
success_percentage = (successful_scrapes / total_papers) * 100

# Print the result
print(f"Successfully scraped text from {successful_scrapes} out of {total_papers} papers ({success_percentage:.2f}% success rate)")


/tmp/ipykernel_5232/3042011097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_with_url['full_article_text'] = ""


Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_3.pdf due to an error: cannot open broken document
Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_17.pdf due to an error: cannot open broken document
Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_23.pdf due to an error: cannot open broken document
Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_27.pdf due to an error: cannot open broken document
Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_29.pdf due to an error: cannot open broken document
Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_46.pdf due to an error: cannot open broken document
Skipped processing //workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_51.pdf due to an error: cannot open broken document
Skipped

## Minimally clean text

In [18]:
data_with_full_text = filtered_data[~filtered_data['full_article_text'].isnull()]
data_with_full_text = data_with_full_text.replace(r'\n',' ', regex=True) 
data_with_full_text.head(1)

,PUBLICATION INFORMATION,Author(s),Title,Journal,"Volume, issue, pages",Year,Full text link,PUBMED link,STUDY DESCRIPTION AND METHODS,Study design,...,Facility type,DATA EXTRACTION,Awaiting completion. You may use the above link to download a CSV,Checked out by/date due,Submitted by/date submitted,Submission reviewed by/date reviewed,Unnamed: 214,Unnamed: 215,Downloaded PDF Path,full_article_text
0,,"Al-Kaisy A, Van Buyten JP, Carganillo R, Caraw...","10 kHz SCS therapy for chronic pain, effects o...",Sci Rep,9(1):11441,2019,https://www.nature.com/articles/s41598-019-477...,https://www.ncbi.nlm.nih.gov/pubmed/31391503,,Post hoc analysis of data from two published s...,...,NaN,,NaN,NaN,Anand Rotte/3 Sept 2019,JBS/finalized May 2020,NaN,NaN,/workspaces/Wikistim-Summarization/00_source_d...,1 Scientific RepoRtS | (2019) 9:11441 ...


## Save text to csv

In [19]:
data_with_full_text.to_csv('/workspaces/Wikistim-Summarization/00_source_data/uncleaned_text.csv', index=False)  

## Calculate average token length

In [21]:
paper_data = pd.read_csv('/workspaces/Wikistim-Summarization/00_source_data/uncleaned_text.csv')

def average_token_length(text):
    tokens = word_tokenize(text)
    token_lengths = [len(token) for token in tokens]
    if len(token_lengths) > 0:
        return sum(token_lengths)
    else:
        return 0
    

paper_data['Token_Length'] = paper_data['full_article_text'].apply(average_token_length)
overall_average = paper_data['Token_Length'].mean()
print("Overall Average Token Length:", overall_average)

Overall Average Token Length: 32586.833333333332
